#**Hotel Reviews NLP Project**

####Names: MacKenna Landry, Bryanna Boone, Sergio, Sophia Kurz

###**Data Pre-Processing**

https://www.kaggle.com/datasets/juhibhojani/hotel-reviews

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

hotel_data = pd.read_csv("hotel_reviews.csv")
hotel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7001 entries, 0 to 7000
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Index              7001 non-null   int64  
 1   Name               7001 non-null   object 
 2   Area               7001 non-null   object 
 3   Review_Date        7001 non-null   object 
 4   Rating_attribute   7001 non-null   object 
 5   Rating(Out of 10)  7001 non-null   float64
 6   Review_Text        6994 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 383.0+ KB


In [ ]:
hotel_data.head()

,Index,Name,Area,Review_Date,Rating_attribute,Rating(Out of 10),Review_Text
0,0,Hotel The Pearl,"Paharganj, New Delhi",Jul-23,Best budget friendly hotel,9.0,Hotel the pearl is perfect place to stay in De...
1,1,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Amazing place,9.0,Location of the hotel is perfect. The hotel is...
2,2,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Overall good stay. Economic.,9.0,"Location, Indian food."
3,3,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Lovely,9.0,The location and the hotel itself is great. Ne...
4,4,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Great hotel Great staff and great staying,9.0,Friendly and smiling staffs.. The reception st...


In [ ]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_and_tokenize_text(text):
    if pd.isnull(text):
        return ''

    # Tokenization using regular expression
    tokens = re.findall(r'\b\w+\b', text.lower())

    # Stopword removal and lemmatization
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(clean_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
hotel_data['Rating_attribute'] = hotel_data['Rating_attribute'].apply(clean_and_tokenize_text)
hotel_data['Review_Text'] = hotel_data['Review_Text'].apply(clean_and_tokenize_text)

In [ ]:
hotel_data.head()

,Index,Name,Area,Review_Date,Rating_attribute,Rating(Out of 10),Review_Text
0,0,Hotel The Pearl,"Paharganj, New Delhi",Jul-23,best budget friendly hotel,9.0,hotel pearl perfect place stay delhi paharganj...
1,1,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,amazing place,9.0,location hotel perfect hotel peaceful nice sta...
2,2,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,overall good stay economic,9.0,location indian food
3,3,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,lovely,9.0,location hotel great next time stay nice room ...
4,4,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,great hotel great staff great staying,9.0,friendly smiling staff reception staff excelle...


###**Sentiment Analysis using VADER**

In [ ]:
pip install vaderSentiment

In [ ]:
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

def get_sentiment_label(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'


In [ ]:
#Calculate sentiment score for each review
hotel_data['Sentiment_value'] = hotel_data['Review_Text'].apply(sentiment_scores)


In [ ]:
hotel_data.head()

,Index,Name,Area,Review_Date,Rating_attribute,Rating(Out of 10),Review_Text,Sentiment_value
0,0,Hotel The Pearl,"Paharganj, New Delhi",Jul-23,best budget friendly hotel,9.0,hotel pearl perfect place stay delhi paharganj...,0.9633
1,1,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,amazing place,9.0,location hotel perfect hotel peaceful nice sta...,0.9545
2,2,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,overall good stay economic,9.0,location indian food,0.0000
3,3,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,lovely,9.0,location hotel great next time stay nice room ...,0.9423
4,4,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,great hotel great staff great staying,9.0,friendly smiling staff reception staff excelle...,0.9750


In [ ]:

# Group the DataFrame by hotel name and calculate the average sentiment score for each hotel
hotel_sentiment = hotel_data.groupby('Name').agg({'Sentiment_value': 'mean', 'Review_Text': 'first'}).reset_index()

# Sort the hotels based on their average sentiment scores to rank them
ranked_hotels = hotel_sentiment.sort_values(by='Sentiment_value', ascending=False)


In [ ]:
sentiment_df = pd.DataFrame({
    'Hotel Name': ranked_hotels['Name'],
    'Sentiment Label': ranked_hotels['Sentiment_value'].apply(get_sentiment_label),
    'Sentiment Score': ranked_hotels['Sentiment_value'],
    'Review Text': ranked_hotels['Review_Text']
})

In [ ]:
num_rows = sentiment_df.shape[0]
print("Number of Hotels Reviewed: " + str(num_rows))
print()
print("Examples of the highest ranked hotels based on their sentiment scores:")
sentiment_df[:5]
print("Examples of the lowest ranked hotels based on their sentiment scores:")
sentiment_df[565:]


Number of Hotels Reviewed: 570

Examples of the highest ranked hotels based on their sentiment scores:


,Hotel Name,Sentiment Label,Sentiment Score,Review Text
7,Abuzz Oxfordcaps ShahpurJaat Delhi,Positive,0.9521,property shahpurjat girl property serf veg foo...
192,Hotel City Plaza,Positive,0.9359,lovely stay room comfy clean service every day...
158,Homlee Saket Basement Flat with Bathroom at Gr...,Positive,0.9313,place convenient location wise perfect planing...
477,Roomshala 120 Hotel Crown Near Signature Bridg...,Positive,0.9300,great experience roomshala hotel crown great s...
140,HOTEL CITY INN,Positive,0.9246,pocket friendly hotel good staff service clean...


Examples of the lowest ranked hotels based on their sentiment scores:


,Hotel Name,Sentiment Label,Sentiment Score,Review Text
56,Collection O The Pinaki Residency Near Dlf Ave...,Negative,-0.340125,dirty bed sheet bad odour room
569,noratan palace,Negative,-0.361533,bad condition toilet
534,Townhouse 1251 The Royce By Dsr,Negative,-0.440400,hotel operational quite time still listed even...
34,Bewafa Palace,Negative,-0.762500,let check left road midnight worst experience ...
524,The New Anand Near Karol Bagh Metro Station,Negative,-0.972900,booking com representative respond call denyin...


###**Sentence Embeddings to Find Similar Hotels to Query**

In [ ]:
# partition sentiment df to only include positive reviews to recommend to users
recommend_df = sentiment_df[:450]
recommend_df

,Hotel Name,Sentiment Label,Sentiment Score,Review Text
7,Abuzz Oxfordcaps ShahpurJaat Delhi,Positive,0.952100,property shahpurjat girl property serf veg foo...
192,Hotel City Plaza,Positive,0.935900,lovely stay room comfy clean service every day...
158,Homlee Saket Basement Flat with Bathroom at Gr...,Positive,0.931300,place convenient location wise perfect planing...
477,Roomshala 120 Hotel Crown Near Signature Bridg...,Positive,0.930000,great experience roomshala hotel crown great s...
140,HOTEL CITY INN,Positive,0.924600,pocket friendly hotel good staff service clean...
...,...,...,...,...
54,Collection O Radisson Blue Hotel Nearby Khajur...,Positive,0.110100,red light area
241,Hotel Jai Deluxe- Paharganj New Delhi Railway ...,Positive,0.109860,staff nice helpful
391,OYO Flagship 75287 Pubg Residency,Positive,0.106217,location good near metro station
242,Hotel Jashn Inn near Apollo Hospital,Positive,0.101575,breakfast excellent location also excellent


In [ ]:
reviews = recommend_df["Review Text"]

reviews[:5]

7      property shahpurjat girl property serf veg foo...
192    lovely stay room comfy clean service every day...
158    place convenient location wise perfect planing...
477    great experience roomshala hotel crown great s...
140    pocket friendly hotel good staff service clean...
Name: Review Text, dtype: object

In [ ]:
#user input
query = input()
query = str(query)

good views


In [ ]:
print(query)

good views


Word2Vec

In [ ]:
import gensim.downloader as api

# Load the pre-trained Word2Vec model

# Download a pre-trained word2vec (trained on Google News data)
w2v_model = api.load("word2vec-google-news-300")

# Compute similarity between words
def compute_similarity(model, word1, word2):
    try:
        return model.similarity(word1, word2)
    except KeyError:
        return 0  # return 0 if either of the words is not in the vocabulary

In [ ]:
# Function to extract sentence vector from word vectors by averaging word embeddings
from scipy.spatial.distance import cosine
import numpy as np

def extract_sentence_vector(sentence):
    words = sentence.split()
    word_vectors = [w2v_model[word] for word in words if word in w2v_model]
    if not word_vectors:
        return None  # Return None if no word vectors are found
    sentence_vector = np.mean(word_vectors, axis=0)
    return sentence_vector

def similarity (x1, x2):
  # similarity is the opposite of distance
  return 1 - cosine(x1, x2)

In [ ]:
# Example of extracting sentence vector using mean of word vectors

# change hotel names to list so that we can get hotel recs by index nums
hotel_list = recommend_df["Hotel Name"].tolist()

#process dataset to get a list of sentencese (the reviews)
review_list = reviews.tolist()#lowercase and append to list
print(review_list[:5])
#extract sentence vectors and store in list
vector_list = []
for review in review_list:
  sentence_vector = extract_sentence_vector(review)
  vector_list.append(sentence_vector)

['property shahpurjat girl property serf veg food taste really amazing washroom dorm really clean hygenic bed really cozy clean privacy curtain 24hrs ac 3time meal included 170 rupee nice cabin room dorm accommodate 1person looking privacy long stay overall priyanka warden property soo sweet decent book property without second thought totally worth', 'lovely stay room comfy clean service every day receptionist team friendly professional breakfast service nice fast would highly recommend hotel', 'place convenient location wise perfect planing go mehrauli pretty often place stay loved hospitality', 'great experience roomshala hotel crown great stay good location property clean hygienic', 'pocket friendly hotel good staff service clean room near metro station hence help easy commute various place']


In [ ]:
query = query.lower()
#print(query)
sentence_vector2 = extract_sentence_vector(query)
#check vector shape
print(sentence_vector2.shape)

#compute all similarities
similarity_list = []
new_hotel = []
i = 0
for vector in vector_list:
  if vector is not None:
    sim = similarity(vector, sentence_vector2)
    similarity_list.append(sim)
    new_hotel.append(hotel_list[i])

  i +=1

print("Some similarity scores", similarity_list[:5])

#hotel_list and similarity_list are parallel - create dictionary
hotel_dict = {}
for i, hotel in enumerate(new_hotel):
  hotel_dict[hotel] = similarity_list[i]


sorted_hotel = dict(sorted(hotel_dict.items(), key=lambda item: item[1]))
#print(len(sorted_hotel))

top_hotel = list(sorted_hotel.keys())[444:]
reviews =  list(sorted_hotel.values())[444:]
num = 1
for i in range(4, -1, -1):
  print(f"{num}. {top_hotel[i]}: {reviews[i]}")
  num+=1

(300,)
Some similarity scores [0.44318947196006775, 0.42680802941322327, 0.43238067626953125, 0.5177114605903625, 0.3812007009983063]
1. Le TRESOR: 0.6298931837081909
2. Hotel Amar Inn- Lajpat Nagar Central Market: 0.6209972500801086
3. Anchal Hotel: 0.604943573474884
4. Hotel Stay Villa - Couple Friendly - 10 mints from Chattarpur metro station: 0.5939834117889404
5. Hotel New City Lite Near IGI Airport Delhi: 0.593643307685852


Using BERT

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# change hotel names to list so that we can get hotel recs by index nums
hotel_list2 = recommend_df["Hotel Name"].to_list()

# Load a pre-trained BERT model
model = SentenceTransformer('bert-base-uncased')

#process dataset to get a list of sentencese (the reviews)
review_list2 = []
for review in review_list:
  review_list2.append(review) #lowercase and append to list

print(review_list2[:5])

sentence_embeddings = model.encode(review_list2)

query_embeddings = model.encode([query])
#check vector
print(query_embeddings.shape)

# Compute similarity between the first two sentences
similarity2 = []
for embed in sentence_embeddings:
  similarity_score = similarity(embed,query_embeddings[0])
  similarity2.append(similarity_score)

['property shahpurjat girl property serf veg food taste really amazing washroom dorm really clean hygenic bed really cozy clean privacy curtain 24hrs ac 3time meal included 170 rupee nice cabin room dorm accommodate 1person looking privacy long stay overall priyanka warden property soo sweet decent book property without second thought totally worth', 'lovely stay room comfy clean service every day receptionist team friendly professional breakfast service nice fast would highly recommend hotel', 'place convenient location wise perfect planing go mehrauli pretty often place stay loved hospitality', 'great experience roomshala hotel crown great stay good location property clean hygienic', 'pocket friendly hotel good staff service clean room near metro station hence help easy commute various place']
(1, 768)


In [ ]:
#hotel_list and similarity_list are parallel - create dictionary
hotel_dict2 = {}
for i,hotel in enumerate(hotel_list2):
  hotel_dict2[hotel] = similarity2[i]


sorted_hotel2 = dict(sorted(hotel_dict2.items(), key=lambda item: item[1]))

top_hotel2 = list(sorted_hotel2.keys())[444:]
reviews2 =  list(sorted_hotel2.values())[444:]
#printing out top 5 most similar hotels
print("Best hotels for you:")
num = 1
for i in range(4, -1, -1):
  print(f"{num}. {top_hotel2[i]}: {reviews2[i]}")
  num+=1

Best hotels for you:
1. FabHotel Grand Stay: 0.7884588241577148
2. HOTEL AMBIENCE DX PAHARGANJ NEW Delhi NEAR Rk ASHRAM METRO STATION: 0.7839747071266174
3. Daffodils inn: 0.7827611565589905
4. Hotel Airport Residency: 0.7772313356399536
5. Hotel Golden Palace By Otaexpertz: 0.7772313356399536


###**Evaluation**

In [ ]:
recommend_df.to_csv('positive_hotels.csv')

def extract_sentence_vector(sentence, model=w2v_model):
    words = sentence.split()
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

def get_top_k_recommendations(query, reviews, hotel_names, sentiment_scores, model, k=5):
    query_vector = extract_sentence_vector(query.lower(), model)
    similarities = [
        1 - cosine(query_vector, extract_sentence_vector(review.lower(), model)) if review is not None else 0
        for review in reviews
    ]
    combined_scores = [sim * score for sim, score in zip(similarities, sentiment_scores)]
    sorted_indices = np.argsort(combined_scores)[::-1]
    top_k_indices = sorted_indices[:k]
    return [hotel_names[idx] for idx in top_k_indices]

sentiment_scores = recommend_df["Sentiment Score"].tolist()

def evaluate_model(queries_and_expected_results, reviews, hotel_names, model, ks=[1, 3, 5]):
    results = {k: {'precision': [], 'recall': [], 'f1': []} for k in ks}
    for query, expected in queries_and_expected_results.items():
        for k in ks:
            recommended = get_top_k_recommendations(query, reviews, hotel_names, sentiment_scores, model, k)
            true_positives = len(set(recommended) & set(expected))
            precision = true_positives / k
            recall = true_positives / len(expected)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
            results[k]['precision'].append(precision)
            results[k]['recall'].append(recall)
            results[k]['f1'].append(f1)
    return results


reviews = recommend_df["Review Text"]
hotel_names = recommend_df["Hotel Name"].to_list()

queries_and_expected_results = {
    "Hotels with friendly staff": ["HOTEL CITY INN", "Hotel Cosmo - Karol Bagh", "HOTEL PARK PRIDE","Pamposh Guest House GK II Market","FabHotel Cameron","Hotel Airport JMD Divine 15 MINTUE IGI AIRPORT DELHI"],
    "Hotels with excellent breakfast": ["OYO Royal Galaxy", "Hotel New City Lite Near IGI Airport Delhi", "Hotel Uptown - By Srida Hotels","Hotel City Plaza","Hotel Le-Mount Near IGI Airport Delhi"],
    "Comfortable Hotels": ["Hotel The Pearl", "HOTEL PARK PRIDE", "The Neem", "Own Stay", "Woodpecker Apartments Hauz khas"],
    "Clean hotels": ["Hotel Cosmo - Karol Bagh", "HOTEL CITY INN", "Hotel City Plaza", "Hotel Admire Gold - Pachim Vihar Delhi", "Townhouse 712 Sector 24 Rohini Near Rithala Metro Station"],
    "Budget friendly Hotels": ["Hotel Lotus Stay At Delhi Airport", "Airport Hotel IVY", "Amax Inn", "Qotel Jasola Vihar", "Hotel Uptown - By Srida Hotels"],
    "Hotels with a pool": ["Aloft New Delhi Aerocity", "Park Plaza Shahdara", "Atrio by Devam", "The Lodhi â€“ A member of The Leading Hotels Of The World"],
    "Clean hotels with great service": ["Hotel City Plaza", "HOTEL CITY INN","Hotel Cosmo - Karol Bagh","Hotel The Pearl","Hotel Sky wood At Airport"],
    "Hotels that are safe, quiet and comfortable": ["The Manor - New Delhi", "Bloomrooms @ New Delhi Railway Station", "FabHotel La Paz Stay B&B", "Sarthak Palace", "Hotel Grand Park Inn"],
}

word2vec_results = evaluate_model(queries_and_expected_results, reviews, hotel_names, w2v_model)

print("Word2Vec Results:")
for query, expected in queries_and_expected_results.items():
    print(f"\nQuery: {query}")
    for k in sorted(word2vec_results.keys()):
        print(f"\n  For K={k}:")
        precision = word2vec_results[k]['precision'][list(queries_and_expected_results.keys()).index(query)]
        recall = word2vec_results[k]['recall'][list(queries_and_expected_results.keys()).index(query)]
        f1 = word2vec_results[k]['f1'][list(queries_and_expected_results.keys()).index(query)]
        print(f"    Precision: {precision}")
        print(f"    Recall: {recall}")

/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


Word2Vec Results:

Query: Hotels with friendly staff

  For K=1:
    Precision: 1.0
    Recall: 0.16666666666666666

  For K=3:
    Precision: 0.3333333333333333
    Recall: 0.16666666666666666

  For K=5:
    Precision: 0.6
    Recall: 0.5

Query: Hotels with excellent breakfast

  For K=1:
    Precision: 1.0
    Recall: 0.2

  For K=3:
    Precision: 0.6666666666666666
    Recall: 0.4

  For K=5:
    Precision: 0.4
    Recall: 0.4

Query: Comfortable Hotels

  For K=1:
    Precision: 1.0
    Recall: 0.2

  For K=3:
    Precision: 0.6666666666666666
    Recall: 0.4

  For K=5:
    Precision: 0.4
    Recall: 0.4

Query: Clean hotels

  For K=1:
    Precision: 0.0
    Recall: 0.0

  For K=3:
    Precision: 0.3333333333333333
    Recall: 0.2

  For K=5:
    Precision: 0.6
    Recall: 0.6

Query: Budget friendly Hotels

  For K=1:
    Precision: 0.0
    Recall: 0.0

  For K=3:
    Precision: 0.0
    Recall: 0.0

  For K=5:
    Precision: 0.2
    Recall: 0.2

Query: Hotels with a pool

  F

In [ ]:
def extract_sentence_vector_bert(sentence, model):
    return model.encode([sentence])[0]

def get_top_k_recommendations(query, reviews, hotel_names, sentiment_scores, model, k=5):
    query_vector = extract_sentence_vector_bert(query.lower(), model)
    similarities = [
        1 - cosine(query_vector, extract_sentence_vector_bert(review.lower(), model)) if review is not None else 0
        for review in reviews
    ]
    combined_scores = [sim * score for sim, score in zip(similarities, sentiment_scores)]
    sorted_indices = np.argsort(combined_scores)[::-1]
    top_k_indices = sorted_indices[:k]
    return [hotel_names[idx] for idx in top_k_indices]

sentiment_scores = recommend_df["Sentiment Score"].tolist()

def evaluate_model(queries_and_expected_results, reviews, hotel_names, model, ks=[1, 3, 5]):
    results = {k: {'precision': [], 'recall': [], 'f1': []} for k in ks}
    for query, expected in queries_and_expected_results.items():
        for k in ks:
            recommended = get_top_k_recommendations(query, reviews, hotel_names, sentiment_scores, model, k)
            true_positives = len(set(recommended) & set(expected))
            precision = true_positives / k
            recall = true_positives / len(expected)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
            results[k]['precision'].append(precision)
            results[k]['recall'].append(recall)
            results[k]['f1'].append(f1)
    return results

bert_results = evaluate_model(queries_and_expected_results, reviews, hotel_names, model=model)

print("BERT Results:")
for query, expected in queries_and_expected_results.items():
    print(f"\nQuery: {query}")
    for k in sorted(bert_results.keys()):
        print(f"\n  For K={k}:")
        precision = bert_results[k]['precision'][list(queries_and_expected_results.keys()).index(query)]
        recall = bert_results[k]['recall'][list(queries_and_expected_results.keys()).index(query)]
        f1 = bert_results[k]['f1'][list(queries_and_expected_results.keys()).index(query)]
        print(f"    Precision: {precision}")
        print(f"    Recall: {recall}")

BERT Results:

Query: Hotels with friendly staff

  For K=1:
    Precision: 1.0
    Recall: 0.16666666666666666
    F1 Score: 0.2857142857142857

  For K=3:
    Precision: 0.3333333333333333
    Recall: 0.16666666666666666
    F1 Score: 0.2222222222222222

  For K=5:
    Precision: 0.4
    Recall: 0.3333333333333333
    F1 Score: 0.3636363636363636

Query: Hotels with excellent breakfast

  For K=1:
    Precision: 1.0
    Recall: 0.2
    F1 Score: 0.33333333333333337

  For K=3:
    Precision: 0.3333333333333333
    Recall: 0.2
    F1 Score: 0.25

  For K=5:
    Precision: 0.2
    Recall: 0.2
    F1 Score: 0.20000000000000004

Query: Comfortable Hotels

  For K=1:
    Precision: 0.0
    Recall: 0.0
    F1 Score: 0

  For K=3:
    Precision: 0.0
    Recall: 0.0
    F1 Score: 0

  For K=5:
    Precision: 0.0
    Recall: 0.0
    F1 Score: 0

Query: Clean hotels

  For K=1:
    Precision: 0.0
    Recall: 0.0
    F1 Score: 0

  For K=3:
    Precision: 0.0
    Recall: 0.0
    F1 Score: 0

  